# 数据格式处理：JSON 和 XML

## 学习目标
- 掌握 JSON 和 XML 两种主要数据交换格式
- 学会使用 Python 原生库处理这两种格式
- 了解在 Web 开发中的实际应用场景
- 掌握数据序列化和反序列化的最佳实践

## 前言
在现代 Web 开发中，数据交换格式是后端开发的核心技能。JSON 因其轻量级和易读性成为了 API 开发的首选，而 XML 在企业级应用和配置文件中仍然广泛使用。


## 1. JSON 数据格式

### 什么是 JSON？
JSON (JavaScript Object Notation) 是一种轻量级的数据交换格式。尽管名字中有 JavaScript，但它是语言无关的。

### JSON 的特点
- **轻量级**：相比 XML 更简洁
- **易读性**：人类可读，结构清晰
- **原生支持**：现代编程语言都有内置支持
- **Web 友好**：浏览器原生支持

### JSON 数据类型
- `string`：字符串
- `number`：数字
- `boolean`：布尔值
- `null`：空值
- `object`：对象（键值对）
- `array`：数组


In [1]:
import json
from typing import Dict, List, Any, Optional
import logging
from datetime import datetime
from dataclasses import dataclass, asdict
from decimal import Decimal

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 示例 JSON 数据
sample_json = """
{
    "user_id": 12345,
    "username": "john_doe",
    "email": "john@example.com",
    "is_active": true,
    "balance": 1500.50,
    "created_at": "2024-01-15T10:30:00Z",
    "profile": {
        "first_name": "John",
        "last_name": "Doe",
        "age": 30,
        "address": {
            "street": "123 Main St",
            "city": "New York",
            "country": "USA"
        }
    },
    "interests": ["python", "backend", "api"],
    "metadata": null
}
"""

print("示例 JSON 数据:")
print(sample_json)


示例 JSON 数据:

{
    "user_id": 12345,
    "username": "john_doe",
    "email": "john@example.com",
    "is_active": true,
    "balance": 1500.50,
    "created_at": "2024-01-15T10:30:00Z",
    "profile": {
        "first_name": "John",
        "last_name": "Doe",
        "age": 30,
        "address": {
            "street": "123 Main St",
            "city": "New York",
            "country": "USA"
        }
    },
    "interests": ["python", "backend", "api"],
    "metadata": null
}



### JSON 反序列化（解析）

**概念**：将 JSON 字符串转换为 Python 对象的过程


In [ ]:
# 1. 基本的 JSON 解析
def parse_json_basic(json_string: str) -> Dict[str, Any]:
    """
    基础 JSON 解析方法
    
    Args:
        json_string: JSON 格式的字符串
        
    Returns:
        解析后的 Python 字典
        
    Raises:
        json.JSONDecodeError: JSON 格式错误时抛出
    """
    try:
        data = json.loads(json_string)
        logger.info(f"成功解析 JSON，数据类型: {type(data)}")
        return data
    except json.JSONDecodeError as e:
        logger.error(f"JSON 解析失败: {e}")
        raise

# 解析示例数据
user_data = parse_json_basic(sample_json)
print("解析后的数据类型:", type(user_data))
print("用户 ID:", user_data['user_id'])
print("用户名:", user_data['username'])
print("嵌套数据 - 姓名:", user_data['profile']['first_name'])
print("数组数据 - 兴趣:", user_data['interests'])


In [2]:
# 2. 安全的 JSON 解析（生产环境推荐）
def parse_json_safe(json_string: str, default_value: Any = None) -> Any:
    """
    安全的 JSON 解析，带有错误处理和默认值
    
    Args:
        json_string: JSON 格式的字符串
        default_value: 解析失败时返回的默认值
        
    Returns:
        解析后的数据或默认值
    """
    if not json_string or not json_string.strip():
        logger.warning("输入的 JSON 字符串为空")
        return default_value
    
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        logger.error(f"JSON 解析失败: {e.msg} at line {e.lineno} column {e.colno}")
        return default_value
    except Exception as e:
        logger.error(f"解析过程中发生未知错误: {e}")
        return default_value

# 测试安全解析
print("=== 安全解析测试 ===")
print("正常数据:", parse_json_safe(sample_json))
print("空字符串:", parse_json_safe("", {"error": "empty"}))
print("错误格式:", parse_json_safe('{"invalid": json}', {"error": "invalid"}))


ERROR:__main__:JSON 解析失败: Expecting value at line 1 column 13


=== 安全解析测试 ===
正常数据: {'user_id': 12345, 'username': 'john_doe', 'email': 'john@example.com', 'is_active': True, 'balance': 1500.5, 'created_at': '2024-01-15T10:30:00Z', 'profile': {'first_name': 'John', 'last_name': 'Doe', 'age': 30, 'address': {'street': '123 Main St', 'city': 'New York', 'country': 'USA'}}, 'interests': ['python', 'backend', 'api'], 'metadata': None}
空字符串: {'error': 'empty'}
错误格式: {'error': 'invalid'}


### JSON 序列化（生成）

**概念**：将 Python 对象转换为 JSON 字符串的过程


In [3]:
# JSON 序列化示例

# 1. 基本序列化
python_data = {
    "id": 1,
    "name": "Python Backend Course",
    "price": 299.99,
    "active": True,
    "tags": ["python", "backend", "api"],
    "instructor": {
        "name": "PyBackendPro",
        "experience": 10
    },
    "created_at": "2024-01-15T10:30:00Z"
}

# 基本转换
json_string = json.dumps(python_data)
print("基本序列化:")
print(json_string)
print()

# 2. 格式化输出（推荐用于调试）
json_formatted = json.dumps(python_data, indent=2, ensure_ascii=False)
print("格式化序列化:")
print(json_formatted)
print()

# 3. 生产环境序列化（紧凑格式）
json_compact = json.dumps(python_data, separators=(',', ':'), ensure_ascii=False)
print("紧凑序列化:")
print(json_compact)


基本序列化:
{"id": 1, "name": "Python Backend Course", "price": 299.99, "active": true, "tags": ["python", "backend", "api"], "instructor": {"name": "PyBackendPro", "experience": 10}, "created_at": "2024-01-15T10:30:00Z"}

格式化序列化:
{
  "id": 1,
  "name": "Python Backend Course",
  "price": 299.99,
  "active": true,
  "tags": [
    "python",
    "backend",
    "api"
  ],
  "instructor": {
    "name": "PyBackendPro",
    "experience": 10
  },
  "created_at": "2024-01-15T10:30:00Z"
}

紧凑序列化:
{"id":1,"name":"Python Backend Course","price":299.99,"active":true,"tags":["python","backend","api"],"instructor":{"name":"PyBackendPro","experience":10},"created_at":"2024-01-15T10:30:00Z"}


In [5]:
# 自定义 JSON 编码器（处理特殊数据类型）
class CustomJSONEncoder(json.JSONEncoder):
    """
    自定义 JSON 编码器，处理 datetime、Decimal 等特殊类型
    """
    def default(self,obj):
        if isinstance(obj,datetime):
            return obj.isoformat()
        elif isinstance(obj,Decimal):
            return float(obj)
        elif hasattr(obj,'__dict__'):
            return obj.__dict__
        return super().default(obj)

# 使用数据类
@dataclass
class Product:
    id: int
    name: str
    price: Decimal
    created_at: datetime
    
    def to_dict(self) -> Dict[str, Any]:
        """转换为字典格式"""
        return asdict(self)

# 创建包含特殊类型的数据
product = Product(
    id=1,
    name="Python Course",
    price=Decimal('299.99'),
    created_at=datetime.now()
)

print("=== 自定义编码器示例 ===")
try:
    # 使用自定义编码器
    json_with_custom = json.dumps(product, cls=CustomJSONEncoder, indent=2)
    print("自定义编码器结果:")
    print(json_with_custom)
except Exception as e:
    print(f"编码失败: {e}")

# 使用 to_dict 方法
print("\n使用 to_dict 方法:")
product_dict = product.to_dict()
json_from_dict = json.dumps(product_dict, default=str, indent=2)
print(json_from_dict)


=== 自定义编码器示例 ===
自定义编码器结果:
{
  "id": 1,
  "name": "Python Course",
  "price": 299.99,
  "created_at": "2025-06-16T08:42:37.939143"
}

使用 to_dict 方法:
{'id': 1, 'name': 'Python Course', 'price': Decimal('299.99'), 'created_at': datetime.datetime(2025, 6, 16, 8, 42, 37, 939143)}
{
  "id": 1,
  "name": "Python Course",
  "price": "299.99",
  "created_at": "2025-06-16 08:42:37.939143"
}


## 2. XML 数据格式

### 什么是 XML？
XML (eXtensible Markup Language) 是一种标记语言，用于存储和传输数据。

### XML 的特点
- **自描述性**：标签名称描述数据内容
- **层次结构**：树形结构，支持嵌套
- **扩展性**：可以自定义标签
- **验证支持**：可以使用 DTD 或 XSD 验证

### XML vs JSON
| 特性 | XML | JSON |
|------|-----|------|
| 可读性 | 较好 | 更好 |
| 体积 | 较大 | 较小 |
| 解析速度 | 较慢 | 较快 |
| 数据类型 | 字符串 | 多种原生类型 |
| 验证 | 强大 | 基础 |


In [6]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import xml.parsers.expat
from typing import Dict, List

# 示例 XML 数据
sample_xml = """<?xml version="1.0" encoding="UTF-8"?>
<users>
    <user id="1" status="active">
        <profile>
            <username>john_doe</username>
            <email>john@example.com</email>
            <first_name>John</first_name>
            <last_name>Doe</last_name>
            <age>30</age>
        </profile>
        <address>
            <street>123 Main St</street>
            <city>New York</city>
            <country>USA</country>
        </address>
        <interests>
            <interest>python</interest>
            <interest>backend</interest>
            <interest>api</interest>
        </interests>
        <created_at>2024-01-15T10:30:00Z</created_at>
    </user>
    <user id="2" status="inactive">
        <profile>
            <username>jane_smith</username>
            <email>jane@example.com</email>
            <first_name>Jane</first_name>
            <last_name>Smith</last_name>
            <age>28</age>
        </profile>
        <address>
            <street>456 Oak Ave</street>
            <city>Los Angeles</city>
            <country>USA</country>
        </address>
        <interests>
            <interest>data_science</interest>
            <interest>machine_learning</interest>
        </interests>
        <created_at>2024-01-10T09:15:00Z</created_at>
    </user>
</users>"""

print("示例 XML 数据:")
print(sample_xml[:500] + "...")  # 显示前500个字符


示例 XML 数据:
<?xml version="1.0" encoding="UTF-8"?>
<users>
    <user id="1" status="active">
        <profile>
            <username>john_doe</username>
            <email>john@example.com</email>
            <first_name>John</first_name>
            <last_name>Doe</last_name>
            <age>30</age>
        </profile>
        <address>
            <street>123 Main St</street>
            <city>New York</city>
            <country>USA</country>
        </address>
        <interests>
            <interest>...


In [10]:
# XML 解析示例

def parse_xml_to_dict(xml_string: str) -> Dict[str, Any]:
    """
    将 XML 字符串解析为 Python 字典
    
    Args:
        xml_string: XML 格式的字符串
        
    Returns:
        解析后的字典数据
    """
    try:
        root = ET.fromstring(xml_string)
        
        def element_to_dict(element):
            """递归将 XML 元素转换为字典"""
            result = {}
            
            # 添加属性
            if element.attrib:
                result.update(element.attrib)
            
            # 处理子元素
            children = list(element)
            if children:
                child_dict = {}
                for child in children:
                    child_data = element_to_dict(child)
                    
                    # 处理重复的标签名（转换为列表）
                    if child.tag in child_dict:
                        if not isinstance(child_dict[child.tag], list):
                            child_dict[child.tag] = [child_dict[child.tag]]
                        child_dict[child.tag].append(child_data)
                    else:
                        child_dict[child.tag] = child_data
                
                result.update(child_dict)
            
            # 处理文本内容
            if element.text and element.text.strip():
                if result:  # 如果有其他内容，将文本作为 '_text' 字段
                    result['_text'] = element.text.strip()
                else:  # 如果只有文本，直接返回文本
                    return element.text.strip()
            
            return result
        
        return {root.tag: element_to_dict(root)}
    
    except ET.ParseError as e:
        logger.error(f"XML 解析失败: {e}")
        raise

# 解析 XML 数据
xml_data = parse_xml_to_dict(sample_xml)
print(xml_data)
print(list(xml_data.keys()))
print(xml_data['users']['user'])
print("=== XML 解析结果 ===")
print("根元素:", list(xml_data.keys())[0])
print("用户数量:", len(xml_data['users']['user']))
print("第一个用户的用户名:", xml_data['users']['user'][0]['profile']['username'])
print("第一个用户的属性:", {k: v for k, v in xml_data['users']['user'][0].items() if not isinstance(v, dict)})


{'users': {'user': [{'id': '1', 'status': 'active', 'profile': {'username': 'john_doe', 'email': 'john@example.com', 'first_name': 'John', 'last_name': 'Doe', 'age': '30'}, 'address': {'street': '123 Main St', 'city': 'New York', 'country': 'USA'}, 'interests': {'interest': ['python', 'backend', 'api']}, 'created_at': '2024-01-15T10:30:00Z'}, {'id': '2', 'status': 'inactive', 'profile': {'username': 'jane_smith', 'email': 'jane@example.com', 'first_name': 'Jane', 'last_name': 'Smith', 'age': '28'}, 'address': {'street': '456 Oak Ave', 'city': 'Los Angeles', 'country': 'USA'}, 'interests': {'interest': ['data_science', 'machine_learning']}, 'created_at': '2024-01-10T09:15:00Z'}]}}
['users']
[{'id': '1', 'status': 'active', 'profile': {'username': 'john_doe', 'email': 'john@example.com', 'first_name': 'John', 'last_name': 'Doe', 'age': '30'}, 'address': {'street': '123 Main St', 'city': 'New York', 'country': 'USA'}, 'interests': {'interest': ['python', 'backend', 'api']}, 'created_at': 

## 3. 实践应用场景

### JSON 应用场景
1. **Web API 响应**：RESTful API 的标准数据格式
2. **配置文件**：应用程序配置（比 INI 更结构化）
3. **数据存储**：NoSQL 数据库（MongoDB）
4. **消息队列**：系统间数据传输
5. **前后端通信**：AJAX 请求的标准格式

### XML 应用场景
1. **配置文件**：复杂的配置需求（Spring、Maven）
2. **数据交换**：企业级系统间通信
3. **Web 服务**：SOAP 协议
4. **文档格式**：Office 文档的底层格式
5. **RSS/Atom**：内容聚合格式


In [12]:
# 生产环境最佳实践

class DataFormatHandler:
    """
    数据格式处理器 - 生产环境推荐的封装类
    """
    
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def safe_json_loads(self, json_string: str, default_value: Any = None) -> Any:
        """安全的 JSON 解析"""
        try:
            return json.loads(json_string)
        except (json.JSONDecodeError, TypeError) as e:
            self.logger.error(f"JSON 解析失败: {e}")
            return default_value
    
    def safe_json_dumps(self, data: Any, **kwargs) -> Optional[str]:
        """安全的 JSON 序列化"""
        try:
            # 默认参数
            default_kwargs = {
                'ensure_ascii': False,
                'separators': (',', ':'),  # 紧凑格式
                'default': str  # 处理不可序列化的对象
            }
            default_kwargs.update(kwargs)
            
            return json.dumps(data, **default_kwargs)
        except (TypeError, ValueError) as e:
            self.logger.error(f"JSON 序列化失败: {e}")
            return None
    
    def validate_json_schema(self, data: Dict[str, Any], required_fields: List[str]) -> bool:
        """简单的 JSON 数据验证"""
        try:
            for field in required_fields:
                if field not in data:
                    self.logger.error(f"缺少必需字段: {field}")
                    return False
            return True
        except Exception as e:
            self.logger.error(f"数据验证失败: {e}")
            return False

# 使用示例
handler = DataFormatHandler()

# 测试数据
test_data = {
    "user_id": 123,
    "username": "test_user",
    "email": "test@example.com"
}

# 安全序列化
json_result = handler.safe_json_dumps(test_data, indent=2)
print("安全序列化结果:")
print(json_result)

# 数据验证
required_fields = ["user_id", "username"]
is_valid = handler.validate_json_schema(test_data, required_fields)
print(f"\n数据验证结果: {is_valid}")

# 测试错误情况
invalid_json = '{"invalid": json}'
result = handler.safe_json_loads(invalid_json, {"error": "解析失败"})
print(f"\n错误处理结果: {result}")


ERROR:DataFormatHandler:JSON 解析失败: Expecting value: line 1 column 13 (char 12)


安全序列化结果:
{
  "user_id":123,
  "username":"test_user",
  "email":"test@example.com"
}

数据验证结果: True

错误处理结果: {'error': '解析失败'}


## 4. 性能优化和最佳实践

### JSON 性能优化
1. **使用 ujson**：更快的 JSON 处理库
2. **流式处理**：处理大型 JSON 文件
3. **数据验证**：使用 jsonschema 进行验证
4. **缓存策略**：缓存序列化结果

### XML 性能优化
1. **选择合适的解析器**：SAX vs DOM vs iterparse
2. **命名空间处理**：正确处理 XML 命名空间
3. **内存管理**：避免将整个文档加载到内存

### 安全考虑
- **输入验证**：验证数据格式和内容
- **大小限制**：防止 DoS 攻击
- **编码处理**：正确处理字符编码
- **敏感信息**：避免在日志中暴露敏感数据

## 总结

JSON 和 XML 是现代 Web 开发中不可或缺的数据格式。选择哪种格式取决于具体的应用场景：

- **选择 JSON**：Web API、配置文件、前后端通信
- **选择 XML**：企业集成、复杂配置、需要数据验证的场景

掌握这两种格式的处理技能是 Python 后端开发者的基本要求。
